In [43]:
from langgraph.graph import StateGraph, START , END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [44]:
load_dotenv()

True

In [45]:
import os
api_key = os.environ.get("PERPLEXITY_API_KEY")


In [46]:
model = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.perplexity.ai",
    model="sonar"  # or another supported Perplexity model
)

In [47]:
#create a state

class BlogState(TypedDict):
    title: str
    outline:str
    content:str
    evaluate: int




In [48]:
def create_outline(state: BlogState)-> BlogState:
    #fetch title 
    title = state["title"]
    
    # call LLm gen Outline 
    prompt = f'Generate a detailed outline for the blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state 
    state['outline'] = outline 

    return state


In [49]:
def create_blog(state : BlogState)-> BlogState :
    title = state['title']
    outline = state['outline']

    prompt = f'write a detailed blog on the title -{title} using the following outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [50]:
def evaluate_blog(state:BlogState)-> BlogState:
    outline = state['outline']
    content = state['content']

    prompt = f'with the content of the bolg -{content} , and outlines -{outline} evaluate the blog and give a score out of 100'

    evaluate =model.invoke(prompt).content

    state['evaluate'] = evaluate

    return state

In [51]:
#create a graph 
graph = StateGraph(BlogState)

#add nodes 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog',evaluate_blog)

#add deges 
graph.add_edge(START , 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','evaluate_blog')
graph.add_edge('evaluate_blog',END)

#execute
workflow = graph.compile()



In [52]:
initial_state= {'title': 'Rise of AI In India'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI In India', 'outline': 'Here is a detailed outline for a blog on the topic "Rise of AI in India":\n\n1. **Introduction**\n   - Overview of Artificial Intelligence (AI) and its global significance.\n   - Brief introduction to India’s role in the AI revolution.\n   - Purpose and significance of the blog (to explore India’s AI growth trajectory, infrastructure, investments, and future prospects).\n\n2. **Current Landscape of AI in India**\n   - Market size and growth projections: India’s AI market projected to reach approximately USD 7.8 billion by 2025 with a CAGR of about 26% through 2031[2][5].\n   - Key sectors driving AI adoption: Industrials & Automotive, Healthcare, Retail, Consumer Packaged Goods (CPG), Banking, Financial Services and Insurance (BFSI), and Agri-tech[2][4].\n   - Rise in AI startups and investments: Annual growth in startups by 38.46% from 2019 to 2024; cumulative AI investments hitting USD 2.25 billion by mid-2024[2].\n   - Increase in demand 

In [53]:
print(final_state['outline'])

Here is a detailed outline for a blog on the topic "Rise of AI in India":

1. **Introduction**
   - Overview of Artificial Intelligence (AI) and its global significance.
   - Brief introduction to India’s role in the AI revolution.
   - Purpose and significance of the blog (to explore India’s AI growth trajectory, infrastructure, investments, and future prospects).

2. **Current Landscape of AI in India**
   - Market size and growth projections: India’s AI market projected to reach approximately USD 7.8 billion by 2025 with a CAGR of about 26% through 2031[2][5].
   - Key sectors driving AI adoption: Industrials & Automotive, Healthcare, Retail, Consumer Packaged Goods (CPG), Banking, Financial Services and Insurance (BFSI), and Agri-tech[2][4].
   - Rise in AI startups and investments: Annual growth in startups by 38.46% from 2019 to 2024; cumulative AI investments hitting USD 2.25 billion by mid-2024[2].
   - Increase in demand for AI and Machine Learning professionals by 38%, highli

In [54]:
print(final_state['content'])

Artificial Intelligence (AI) has emerged as a transformative force globally, reshaping industries, economies, and daily life through advanced data-driven decision-making and automation. India, with its vast talent pool and rapidly digitizing economy, has positioned itself as a key player in this AI revolution. This blog explores India’s AI growth — examining the current landscape, government initiatives, infrastructure, sector impact, challenges, and future prospects.

India’s AI market is experiencing robust growth, projected to reach around USD 7.8 billion by 2025, continuing at a compound annual growth rate (CAGR) of about 26% through 2031[2][5]. Industries such as industrials & automotive, healthcare, retail, consumer packaged goods (CPG), banking, financial services and insurance (BFSI), and agri-tech are leading AI adoption due to their vast operational scope and data needs[2][4]. The AI startup ecosystem in India is thriving, with annual startup growth at 38.46% from 2019 to 202

In [56]:
print(final_state["evaluate"])

Based on the content and detailed outline provided for the blog "Rise of AI in India," the blog is well-structured, comprehensive, and backed by up-to-date, authoritative data from credible sources. Here is an evaluation with scoring out of 100 across key criteria:

1. **Content Accuracy and Currency (25/25):**  
The blog uses latest market data and projections (e.g., market size reaching about USD 7.8 billion by 2025, CAGR around 26-27%, investments in data centers surpassing USD 60 billion by 2024) sourced from credible reports such as IMARC, GMI Research, and the UNCTAD 2025 Technology and Innovation Report[1][2][3]. It accurately reflects sectoral AI adoption and government initiatives like IndiaAI Mission and Union Budget 2025 support[1].

2. **Structure and Organization (20/20):**  
The outline is logically segmented into clear, thematic sections progressing from introduction, market landscape, government policy, infrastructure, sector impact, challenges, opportunities, to future